# Question:
With the current *drive* towards back to the office, who actually spends more time commuting to work?
                
**Data and statistics** can help us answer this question.

## Data Sources:
### Commute Time Data: 
![Logo for IPUMS USA](https://www.ipums.org/sites/www.ipums.org/files/project-logo/logo-usa_0.png)

The American Community Survey (ACS) provides detailed commuting data, including average travel times and modes of transportation. This data can be accessed through the U.S. Census Bureau's data portal. 
[IPUMS USA Datasets](https://usa.ipums.org/usa/)

**Key Variables**

- Years **(YEAR)**
    - 2021
    - 2022
    - 2023

- Travel Time to Work **(TRAVEL_TIME)**
    - Measured in Minutes

- Metropolitan Area **(METRO)**
    - 1 =  Not in metropolitan area
    - 2 = In metropolitan area: In central/principal city

- Commute Method **(COMMUTE_MODE)** 
    - 11 = "Auto",
    - 60 = "Walked only",
    - 31 = "Bus"
    - 50 = "Bicycle"


## Libraries Used

- **Pandas** is a powerful data manipulation library providing data structures such as DataFrames that are useful for storing and working with structured data. 
    - We will be using it for easier data cleaning, reshaping and analysis work.

- **Numpy** is a critical package for scientific computing allowing for hgh level mathematics.
    - We will be using it for exp and log functions.

- **Matplotlib** is a data visualization library that helps create plots and charts in python. 
    - We will be using this library to create plots.

- **Scipy** is a library used for statistical analysis and contains a large number of probability distributions and statistical functions. 
    - We will be using this library to obtain the log-normal best fit.

- **Audio-Plot-Lib** is a custom library used to create audio data visualization.
    - We will be using this library to create accessible audio plots of our data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import audio_plot_lib as apl

## Constants

In [ ]:
tranwork_desc = {
    0: "N/A",
    10: "Auto, truck, or van",
    11: "Auto",
    12: "Driver",
    13: "Passenger",
    14: "Truck",
    15: "Van",
    20: "Motorcycle",
    31: "Bus",
    32: "Bus or trolley bus",
    33: "Bus or streetcar",
    34: "Light rail, streetcar, or trolley",
    35: "Streetcar or trolley car",
    36: "Subway or elevated",
    37: "Long-distance train or commuter train",
    38: "Taxicab",
    39: "Ferryboat",
    50: "Bicycle",
    60: "Walked only",
    70: "Other",
    80: "Worked at home" }

metro_desc = { 1: "Not Metropolitan",
               2: "Metropolitan" }

new_col_names = {'TRANTIME':'TRAVEL_TIME', 'TRANWORK':'COMMUTE_MODE', 'TRANWORK_DESC':'COMMUTE_MODE_DESC'}



## Functions

In [ ]:
def lognorm_fit(data):
    """
    Fit the data to a log-normal distribution and returns the statistics.
    
    Parameters:
    -----------
    data : array-like
        The data to fit
        
    Returns:
    --------
    dict
        Dictionary containing fitted parameters and statistics
    """

    # Fit the data to a log-normal distribution
    shape, loc, scale = stats.lognorm.fit(data, floc=0)  # floc=0 is often used to fix the location at 0
    
    # Get the mean and std of the underlying normal distribution
    mu    = np.log(scale)  # mu = log(scale) since scale = exp(mu)
    sigma = shape          # sigma is the shape parameter
    
    # Compute statistics
    mode = np.exp(mu - sigma**2)
    mean = np.exp(mu + (sigma**2/2))
    variance = (mean**2) * (np.exp(sigma**2) - 1)
    
    # Compile statistics
    lognorm_statistics = {
        "Statistic": ["Mu", "Sigma", "Mode", "Mean", "Variance"],
        "Value": [mu, sigma, mode, mean, variance],
        "Parameters": {
            "shape": shape,
            "loc": loc,
            "scale": scale
            }
    }
    
    return lognorm_statistics

In [ ]:
def plot_stacked_log_normals(data, mode_type, min=0, max=200, show=True):
    bins = 20

    title0 = f"Metropolitan {tranwork_desc[mode_type]}"
    title1 = f"Non-Metropolitan {tranwork_desc[mode_type]}"

    titles = [title0, title1]

    metro_data = filter_commute_data(data, is_metropolitan=True, commute_mode_number=mode_type, min_time=min, max_time=max)
    rural_data   = filter_commute_data(data, is_metropolitan=False, commute_mode_number=mode_type, min_time=min, max_time=max)

    metro_fit    = lognorm_fit(metro_data)
    rural_fit      = lognorm_fit(rural_data)

    # Generate a PDF with the fitted parameters
    x = np.linspace(min, max, 1000)
    metro_pdf = stats.lognorm.pdf(x, metro_fit["Parameters"]["shape"], loc=metro_fit["Parameters"]["loc"], scale=metro_fit["Parameters"]["scale"])
    rural_pdf   = stats.lognorm.pdf(x, rural_fit["Parameters"]["shape"], loc=rural_fit["Parameters"]["loc"], scale=rural_fit["Parameters"]["scale"])

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 5))  # 2 rows, 1 column

    ax1.hist(metro_data, bins=bins, density=True, alpha=0.6, color='#f8b8d0', edgecolor='black')
    ax1.plot(x, metro_pdf, label="Log-normal fit", linewidth=3)
    ax1.axvline(metro_fit["Value"][2], color='#b51963', linestyle='--', label=f"Mode ≈ {metro_fit["Value"][2] :.3f}")        # Mode is at index 2
    ax1.axvline(metro_fit["Value"][3], color='#CC79A7', linestyle='--', label=f"Mean ≈ {metro_fit["Value"][3] :.3f}")     # Mean is at index 3
    ax1.set_title(titles[0])
    ax1.set_xlabel("Travel Time")
    ax1.set_ylabel("Density")
    ax1.legend()
    ax1.grid(True)

    ax2.hist(rural_data, bins=bins, density=True, alpha=0.6, color='#8dd2dd', edgecolor='black')
    ax2.plot(x, rural_pdf, label="Log-normal fit", linewidth=3)
    ax2.axvline(rural_fit["Value"][2], color='#b51963', linestyle='--', label=f"Mode ≈ {rural_fit["Value"][2] :.3f}")        # Mode is at index 2
    ax2.axvline(rural_fit["Value"][3], color='#3c4bd1', linestyle='--', label=f"Mean ≈ {rural_fit["Value"][3] :.3f}")     # Mean is at index 3
    ax2.set_title(titles[1])
    ax2.set_xlabel("Travel Time")
    ax2.set_ylabel("Density")
    ax2.legend()
    ax2.grid(True)

    fig.tight_layout()

    if show:
        plt.show()

    metro_text_desc = f"Lognormal fit for {title0} Travel Time with Mode {metro_fit['Value'][2]:.3f} and Mean {metro_fit['Value'][3]:.3f}"
    rural_text_desc   = f"Lognormal fit for {title1} Travel Time with Mode {rural_fit['Value'][2]:.3f} and Mean {rural_fit['Value'][3]:.3f}"

    travel_time_data = np.array([metro_pdf, rural_pdf]).T
    text_desc = [metro_text_desc, rural_text_desc]
    apl.playable.plot(travel_time_data, duration=15, decimals=3, labels=text_desc)

    return fig, (ax1, ax2)


In [ ]:
def filter_commute_data(data, is_metropolitan=True, commute_mode_number=11, min_time=0, max_time=200):
    """
    Filter dataset for commute analysis based on area type and transport mode.
    
    Parameters:
    -----------
    data : array-like
        The data 
    is_metropolitan : Bool
        Boolean value to indicate if we wish to select metropolitan data
    mode : str
        Text to include mode of transportation
    min_time : int
        Integer value representing the minimum time
    max_time: int
        Integer value representing the maximum time
        
    Returns:
    --------
    data:
    filtered data set
    """
    
    metro_value = "Metropolitan" if is_metropolitan else "Not Metropolitan"
    return data[(data['METRO_DESC'] == metro_value) &
                (data['COMMUTE_MODE'] == commute_mode_number) &
                (data['TRAVEL_TIME'] >= min_time) &
                (data['TRAVEL_TIME'] <= max_time)]['TRAVEL_TIME'].copy()

## Load and Preprocess Data

In [ ]:
# Load the data sets
acs_data = pd.read_csv('usa_00006.csv')

# Preprocess
# Map the variable number to a descriptions
acs_data['METRO_DESC']    = acs_data['METRO'].map(metro_desc)
acs_data['TRANWORK_DESC'] = acs_data['TRANWORK'].map(tranwork_desc)

# Rename columns
acs_data.rename(columns=new_col_names, inplace=True)

# Sample of ACS Data with Key Variables

In [ ]:
acs_data[['COMMUTE_MODE_DESC','TRAVEL_TIME','METRO_DESC']].sample(n=10)

## Log-Normal Probability Distribution

The probability density function for the log-normal distribution is defined by two parameters the location $\mu$ and scale $\sigma$ where $x>0$:

$ f(x) = \frac{1}{x \sigma \sqrt{2 \pi} \exp{-\frac{1}{2}} \left( \frac{\ln(x) - \mu}{\sigma} \right)^2} $

## The Maximum Likelihood Estimators for the log-normal distribution

$ \hat{\mu} = \frac{\sum_k \ln{x_k}}{n}$

and

$ \hat{\sigma^2} = \frac{\sum_k (\ln{x_k}- \hat{\mu})^2}{n} $

# Mode and Mean

The mode of a log-normal distribution is the highest point of the curve and is the most likely value to observe.

$ \text{Mode} = \exp{\left( \hat{\mu} - \hat{\sigma^2} \right)} $

The mean of a log-normal distribution is the average of the distribution. For the log-normal distribution the mean is always larger due to the influence of the long right tail.

$ \text{Mean} = \exp{\left( \hat{\mu} + \frac{\hat{\sigma^2}}{2} \right)} $

In [ ]:
for k in [10, 20, 31,50, 60]:
    print("Analysis of ", tranwork_desc[k])
    # Plot the two stacked distributions
    fig, (ax1, ax2) = plot_stacked_log_normals(acs_data, mode_type=k, show=True)